# Trabajo Práctico Integrador 1

## Análisis de datos

#### Integrantes
- Daniel Bazan
- Santiago Casado

##| 1. Dataset seleccionado

### **Conflictos armados en ciudades (Cities and Armed Conflict Events, CACE)**

** explicar el dataset **

** info para descargar desde la api

In [77]:
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, MetaData, text
import json
from io import StringIO

#### a. Descargar el dataset a través de la página oficial

Observando el archivo, se tuvo que agregar el delimitador ";" para poder importarlo correctamente.

In [89]:
path = "dataset/CACE_1989-2017.csv"

# df = pd.read_csv(path, delimiter=';', low_memory=False)
df = pd.read_csv(path, delimiter=';')

/var/folders/_f/7gqqb6190c9b0h_7dkhypsj80000gn/T/ipykernel_1223/2020675959.py:4: DtypeWarning: Columns (8,15,17,18,19,20,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=';')


## 2. Análisis exploratorio de datos

#### a. Columnas y tipos de datos

In [88]:
df.columns

Index(['id', 'active_year', 'type_of_violence', 'conflict_new_id',
       'conflict_name', 'dyad_new_id', 'dyad_name', 'side_a_new_id', 'gwnoa',
       'side_a', 'side_b_new_id', 'gwnob', 'side_b', 'where_prec',
       'where_coordinates', 'City', 'CACE cityname', 'Capital', 'Majorcity',
       'Top3cities', 'Comment', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_id', 'region',
       'event_clarity', 'date_prec', 'year', 'date_start', 'date_end',
       'deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best',
       'low', 'high', 'Unnamed: 42', 'Unnamed: 43'],
      dtype='object')

#### b. Detalles de las columnas del dataset

**Fecha**
| Columna       | Descripción                                                |
| ------------- | ---------------------------------------------------------- |
| `active_year` | Estado del conflicto en ese año (1 = activo, 0 = no activo) |
| `year`        | Año del evento |
| `date_start`  | Fecha de inicio del evento |
| `date_end`    | Fecha de fin del evento |
| `date_prec`   | Precisión de la fecha (1 = exacta, 5 = mas de un mes, menos de un año)|

**Ubicacion**
| Columna             | Descripción                                                          |
| ------------------- | -------------------------------------------------------------------- |
| `where_prec`        | Precisión de la ubicación geográfica (1 = exacta, 3 = incierta) |
| `where_coordinates` | Nombre del lugar del evento |
| `adm_1`             | División administrativa de primer nivel (estado, provincia, etc) |
| `adm_2`             | División administrativa de segundo nivel (municipio, distrito, etc)|
| `latitude`          | Latitud del evento |
| `longitude`         | Longitud del evento |
| `geom_wkt`          | Representación geográfica en formato WKT|
| `priogrid_gid`      | ID de celda espacial de 0.5° x 0.5° (Priogrid)|
| `country`           | Nombre del país |
| `country_id`        | Código del país según Gleditsch & Ward |
| `region`            | Región geográfica amplia (America, Asia, etc.)|
| `City`              | 1 = ocurrió en ciudad (>100.000 hab.), 0 = no |
| `CACE cityname`     | Nombre de la ciudad si es evento urbano según ONU |
| `Capital`           | 1 = ocurrió en capital nacional |
| `Majorcity`         | 1 = ocurrió en ciudad de más de 750.000 habitantes |
| `Top3cities`        | Indica si ocurrió en una de las tres ciudades más grandes del país|

**Entidades participantes**
| Columna            | Descripción                                                     |
| ------------------ | --------------------------------------------------------------- |
| `type_of_violence` | Tipo de violencia: 1 = estatal, 2 = no estatal, 3 = unilateral  |
| `conflict_new_id`  | ID conflicto |
| `conflict_name`    | Nombre del conflicto |
| `dyad_new_id`      | ID de la díada (relación entre dos actores)|
| `dyad_name`        | Par de actores que interactúan entre sí|
| `side_a_new_id`    | ID del actor A |
| `gwnoa`            | Código de país del actor A|
| `side_a`           | Nombre del actor A|
| `side_b_new_id`    | ID del actor B|
| `gwnob`            | Código de país del actor B|
| `side_b`           | Nombre del actor B (puede ser vacío)|

**Víctimas**
| Columna            | Descripción                                          |
| ------------------ | ---------------------------------------------------- |
| `deaths_a`         | Muertes en el actor A |
| `deaths_b`         | Muertes en el actor B |
| `deaths_civilians` | Muertes de civiles |
| `deaths_unknown`   | Muertes no clasificadas |
| `best`             | Estimación central de muertes totales |
| `low`              | Estimación mínima |
| `high`             | Estimación máxima |

**Datos prescindibles**
| Columna         | Descripción                                                            |
| --------------- | ---------------------------------------------------------------------- |
| `id`            | ID general del evento |
| `event_clarity` | Grado de certeza del evento (1 = claro, 3 = dudoso) |
| `Comment`       | Comentarios |
| `Unnamed: 42`   | Columna sin nombre |
| `Unnamed: 43`   | Columna sin nombre |


1. Se revisan datos faltantes o duplicados

In [79]:
print(f"Valores faltantes: {df.isna().sum()}")  # Calcular valores faltantes

Valores faltantes: id                        0
active_year               0
type_of_violence          0
conflict_new_id           0
conflict_name             0
dyad_new_id               0
dyad_name                 0
side_a_new_id             0
gwnoa                 32680
side_a                    0
side_b_new_id             0
gwnob                142160
side_b                    0
where_prec                0
where_coordinates         0
City                      0
CACE cityname        117996
Capital                   0
Majorcity                 0
Top3cities                0
Comment              142696
adm_1                  7237
adm_2                 37225
latitude                  0
longitude                 0
geom_wkt                  0
priogrid_gid              0
country                   0
country_id                0
region                    0
event_clarity             0
date_prec                 0
year                      0
date_start                0
date_end                  0
d

In [80]:
# Calcular la proporción de valores nulos en cada columna
df.isna().mean().round(2)*100

id                     0.0
active_year            0.0
type_of_violence       0.0
conflict_new_id        0.0
conflict_name          0.0
dyad_new_id            0.0
dyad_name              0.0
side_a_new_id          0.0
gwnoa                 23.0
side_a                 0.0
side_b_new_id          0.0
gwnob                 99.0
side_b                 0.0
where_prec             0.0
where_coordinates      0.0
City                   0.0
CACE cityname         83.0
Capital                0.0
Majorcity              0.0
Top3cities             0.0
Comment              100.0
adm_1                  5.0
adm_2                 26.0
latitude               0.0
longitude              0.0
geom_wkt               0.0
priogrid_gid           0.0
country                0.0
country_id             0.0
region                 0.0
event_clarity          0.0
date_prec              0.0
year                   0.0
date_start             0.0
date_end               0.0
deaths_a               0.0
deaths_b               0.0
d

In [81]:
print("Cantidad de datos duplicados:", df.duplicated().sum())

Cantidad de datos duplicados: 0


2. Correción de tipo de datos

La libreria Pandas detecta que mas de una columna tiene datos de diferente tipo. Se identifican cada una y se unifica:

In [82]:
mixed_columns = []

i = 0
for col in df.columns:
    unique_types = df[col].apply(type).unique()
    if len(unique_types) > 1:
        mixed_columns.append(col)
        print(f"\n Columna {i}: {col}")
        print("Tipos encontrados:", unique_types)
        
        #Mostrar 2 ejemplos para cada columna con mas de un tipo de dato
        for dtype in unique_types:
            examples = df[df[col].apply(lambda x: type(x) == dtype)][col].head(2).tolist()
            print(f"  - Tipo {dtype}: {examples}")

    i += 1


 Columna 8: gwnoa
Tipos encontrados: [<class 'float'> <class 'str'>]
  - Tipo <class 'float'>: [678.0, 678.0]
  - Tipo <class 'str'>: ['790', '790']

 Columna 15: City
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 1]
  - Tipo <class 'str'>: ['0', '0']

 Columna 16: CACE cityname
Tipos encontrados: [<class 'str'> <class 'float'>]
  - Tipo <class 'str'>: ['Şan‘ā’', '‘Adan ']
  - Tipo <class 'float'>: [nan, nan]

 Columna 17: Capital
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 18: Majorcity
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 19: Top3cities
Tipos encontrados: [<class 'int'> <class 'str'>]
  - Tipo <class 'int'>: [1, 0]
  - Tipo <class 'str'>: ['0', '0']

 Columna 20: Comment
Tipos encontrados: [<class 'float'> <class 'str'>]
  - Tipo <class 'float'>: [nan, nan]
  - Tipo <class 'str'>: ['Not

In [83]:
# Columnas numéricas
import datetime

df['gwnoa'] = pd.to_numeric(df['gwnoa'], errors='coerce').astype('Int64')

# Columnas binarias
binary_cols = ['City', 'Capital', 'Majorcity', 'Top3cities']

for col in binary_cols:
    # Reemplazar valores conocidos
    df[col] = df[col].replace({
        '0': 0, '1': 1,
        0: 0, 1: 1,
        'True': 1, 'False': 0,
        True: 1, False: 0
    })
    
    # Convertir otros valores a NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Rellenar NaN con 0 o 1 según sea apropiado
    df[col] = df[col].fillna(0).astype('int8')

# Columnas de texto
text_cols = ['conflict_name', 'dyad_name', 'side_a', 'side_b', 'where_coordinates',  'CACE cityname', 'Comment', 'adm_1', 'adm_2', 'geom_wkt', 'country', 'region']
for col in text_cols:
    df[col] = df[col].fillna('').astype(str)

# Columnas de fechas
date_cols = ['date_start', 'date_end']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

print("\nTipos unificados:")
print(df.dtypes)


Tipos unificados:
id                            int64
active_year                   int64
type_of_violence              int64
conflict_new_id               int64
conflict_name                object
dyad_new_id                   int64
dyad_name                    object
side_a_new_id                 int64
gwnoa                         Int64
side_a                       object
side_b_new_id                 int64
gwnob                       float64
side_b                       object
where_prec                    int64
where_coordinates            object
City                           int8
CACE cityname                object
Capital                        int8
Majorcity                      int8
Top3cities                     int8
Comment                      object
adm_1                        object
adm_2                        object
latitude                    float64
longitude                   float64
geom_wkt                     object
priogrid_gid                  int64
country  

3. Eliminación de features.

In [84]:
# Eliminar columnas vacías
columnas_nulas = df.columns[df.isna().all()]

# Eliminarlas
df.drop(columns=columnas_nulas, inplace = True)

print(f"Columnas vacías eliminadas: {columnas_nulas.tolist()}")

Columnas vacías eliminadas: ['Unnamed: 42']


In [85]:
# Eliminar columnas que contengan IDs
columnas_id = ['id', 'conflict_new_id', 'dyad_new_id', 'side_a_new_id', 'side_b_new_id', 'country_id']

for col in columnas_id:
    df.drop(columns=col, inplace = True)

In [86]:
print(df.columns)

Index(['active_year', 'type_of_violence', 'conflict_name', 'dyad_name',
       'gwnoa', 'side_a', 'gwnob', 'side_b', 'where_prec', 'where_coordinates',
       'City', 'CACE cityname', 'Capital', 'Majorcity', 'Top3cities',
       'Comment', 'adm_1', 'adm_2', 'latitude', 'longitude', 'geom_wkt',
       'priogrid_gid', 'country', 'region', 'event_clarity', 'date_prec',
       'year', 'date_start', 'date_end', 'deaths_a', 'deaths_b',
       'deaths_civilians', 'deaths_unknown', 'best', 'low', 'high',
       'Unnamed: 43'],
      dtype='object')
